In [1]:
import warnings
warnings.filterwarnings('ignore')

import torchaudio
import torchaudio.transforms as T
from torch.utils.data import Dataset, DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import loggers as pl_loggers

from audio_classification.data import UrbanSoundDataset
from audio_classification.model import LitDeepCNN, lit_m18, lit_m11

Training on device cuda.


In [2]:
device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
print(f"Training on device {device}")

tb_logger = pl_loggers.TensorBoardLogger('/nfs/homedirs/vashisht/logs/') # chnage username

csv_path = '/nfs/students/winter-term-2020/project-1/datasets/UrbanSound8K/metadata/UrbanSound8K.csv'
file_path = '/nfs/students/winter-term-2020/project-1/datasets/UrbanSound8K/audio/'

# downsample to 8KHz
transform = torchaudio.transforms.Resample(44100, 8000)

# device train_folds and val_folds
folds = list(range(1,11))
val_folds = [10] # CHANGE HERE FOR A DIFFERENT VALIDATION FOLD!
train_folds = [fold for fold in folds if fold not in val_folds] # and fold not in test_folds
#test_folds = [1]

# create train and test sets using chosen transform
train_set = UrbanSoundDataset(csv_path, file_path, train_folds, transform=transform)
val_set = UrbanSoundDataset(csv_path, file_path, val_folds, transform=transform)
print("Train set size: " + str(len(train_set)))
print("val set size: " + str(len(val_set)))

train_loader = DataLoader(train_set, batch_size = 12, shuffle = True, num_workers=20, pin_memory=True)
val_loader = DataLoader(val_set, batch_size = 12, num_workers=20, pin_memory=True)

Training on device cuda
Train set size: 7895
val set size: 837


In [3]:
early_stop_callback = EarlyStopping(
   monitor='val_acc',
   min_delta=0.00,
   patience=5,
   verbose=True,
   mode='max'
)

checkpoint_callback = ModelCheckpoint(
    monitor='val_acc',
    dirpath='/nfs/homedirs/vashisht/weights',
    filename='fold-10-{epoch:02d}-{val_acc:.2f}',
    save_top_k=3,
    mode='max'
)

model = lit_m11(10)
trainer = pl.Trainer(gpus=1, max_epochs=30, progress_bar_refresh_rate=20,
                    callbacks=[checkpoint_callback, early_stop_callback],
                    logger=tb_logger)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [4]:
trainer.fit(model, train_loader, val_loader)

Set SLURM handle signals.

  | Name   | Type       | Params
--------------------------------------
0 | model  | Sequential | 2 M   
1 | linear | Linear     | 5 K   


1

Traceback (most recent call last):
  File "/nfs/homedirs/vashisht/anaconda3/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/nfs/homedirs/vashisht/anaconda3/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/nfs/homedirs/vashisht/anaconda3/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/nfs/homedirs/vashisht/anaconda3/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/nfs/homedirs/vashisht/anaconda3/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/nfs/homedirs/vashisht/anaconda3/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/nfs/homedirs/vashisht/anaconda3/lib/python3

Traceback (most recent call last):
  File "/nfs/homedirs/vashisht/anaconda3/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/nfs/homedirs/vashisht/anaconda3/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/nfs/homedirs/vashisht/anaconda3/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/nfs/homedirs/vashisht/anaconda3/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
